In [1]:
import logging
import os
import sys
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

/Users/madhumithakumar/Documents/google_reviews_tracker/proj_env/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('scrape_google_maps_reviews.log', mode='w'),  # Log to file
        logging.StreamHandler(sys.stdout)  # Log to console (for Jupyter/real-time output)
    ]
)

In [3]:
# def scrape_google_maps_reviews(business_name, city_name, max_locations=5):
#     logging.info("Starting the scrape_google_maps_reviews function")
#     logging.info(f"Business name: {business_name}, City: {city_name}, Max locations: {max_locations}")

#     # Initialize Selenium WebDriver
#     try:
#         options = webdriver.ChromeOptions()
#         options.add_argument("--disable-dev-shm-usage")
#         options.add_argument("--remote-debugging-port=9222")
#         driver = webdriver.Chrome(options=options)
#         driver.maximize_window()
#     except WebDriverException as e:
#         logging.error(f"Failed to initialize WebDriver: {e}")
#         return

#     try:
#         # Open Google Maps
#         logging.info("Opening Google Maps")
#         driver.get("https://www.google.com/maps")
#         wait = WebDriverWait(driver, 20)

#         # Search for the business
#         logging.info("Searching for the business in Google Maps")
#         search_box = wait.until(EC.presence_of_element_located((By.ID, "searchboxinput")))
#         search_box.send_keys(f"{business_name} {city_name}")
#         search_box.send_keys(Keys.RETURN)
#         time.sleep(10)  # Wait for results to load

#         # Find all business listings
#         logging.info("Locating business listings")
#         try:
#             listings = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.Nv2PK')))
#             if not listings:
#                 logging.warning("No business listings found!")
#                 driver.save_screenshot("no_listings_found.png")
#                 return
#         except Exception as e:
#             logging.error(f"Failed to locate business listings: {e}")
#             driver.save_screenshot("error_listings.png")
#             return

#         business_links = []
#         for i, listing in enumerate(listings):
#             if i >= max_locations:
#                 logging.info("Reached max_locations limit. Stopping location collection.")
#                 break
#             try:
#                 link = listing.find_element(By.CSS_SELECTOR, 'a').get_attribute('href')
#                 if "place" in link:
#                     business_links.append(link)
#                     logging.info(f"Found business link: {link}")
#             except Exception as e:
#                 logging.warning(f"Failed to extract business link: {e}")
#                 continue

#         # Data for both CSVs
#         location_data = []  # For overall location details
#         review_data = []  # For individual reviews

#         # Scrape reviews for each business
#         for idx, business_link in enumerate(business_links):
#             logging.info(f"Scraping reviews for business link: {business_link}")
#             driver.get(business_link)
#             time.sleep(5)

#             # Extract details for the first CSV
#             try:
#                 name = driver.find_element(By.CSS_SELECTOR, 'h1').text
#                 address = driver.find_element(By.CSS_SELECTOR, '[data-item-id="address"]').text

#                 # Try to capture the overall rating
#                 try:
#                     overall_rating = driver.find_element(By.CSS_SELECTOR, 'div.F7nice span[aria-hidden="true"]').text
#                 except Exception:
#                     overall_rating = "Rating not found"
#                     logging.warning("Failed to extract overall rating")

#                 # Try to capture the number of reviews
#                 try:
#                     num_reviews_element = driver.find_element(By.CSS_SELECTOR, 'span[aria-label$="reviews"]')
#                     num_reviews_text = num_reviews_element.get_attribute('aria-label')  # Get the full text (e.g., "418 reviews")
#                     num_reviews = int(''.join(filter(str.isdigit, num_reviews_text)))  # Extract numeric value
#                 except Exception:
#                     num_reviews = 0
#                     logging.warning("Failed to extract number of reviews")

#                 location_id = f"LOC_{idx + 1}"  # Generate a unique location ID

#                 location_data.append({
#                     "Location ID": location_id,
#                     "Name": name,
#                     "Address": address,
#                     "Overall Rating": overall_rating,
#                     "Number of Reviews": num_reviews
#                 })
#                 logging.info(f"Captured location details: {name}, {address}, {overall_rating}, {num_reviews}")
#             except Exception as e:
#                 logging.warning(f"Failed to extract location details: {e}")
#                 driver.save_screenshot(f"location_error_{idx}.png")
#                 continue

#             # Click on reviews
#             try:
#                 reviews_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[@role='tab' and contains(@aria-label, 'Reviews')]")))
#                 reviews_button.click()
#                 time.sleep(10)

#                 # Scroll to load all reviews
#                 try:
#                     logging.info("Scrolling to load all reviews")
#                     scrollable_div = driver.find_element(By.CSS_SELECTOR, 'div.m6QErb.DxyBCb.kA9KIf.dS8AEf')  # Updated CSS selector
#                     extracted_reviews = set()  # To track unique reviews
#                     total_reviews_loaded = 0
                
#                     while total_reviews_loaded < num_reviews:  # Continue until all reviews are loaded
#                         # Extract currently loaded reviews
#                         reviews = driver.find_elements(By.CSS_SELECTOR, '.jftiEf')
#                         for review in reviews:
#                             if review not in extracted_reviews:
#                                 try:
#                                     reviewer_name = review.find_element(By.CSS_SELECTOR, '.d4r55').text

#                                     #Try getting review text
#                                     try:
#                                         review_text = review.find_element(By.CSS_SELECTOR, '.MyEned').text
#                                     except Exception:
#                                         review_text = "Review text not found"
#                                         logging.warning("Failed to extract review text")
                                        
#                                     rating = review.find_element(By.CSS_SELECTOR, '.kvMYJc').get_attribute('aria-label')
                
#                                     review_data.append({
#                                         "Location ID": location_id,
#                                         "Reviewer Name": reviewer_name,
#                                         "Review": review_text,
#                                         "Rating": rating
#                                     })
#                                     extracted_reviews.add(review)
#                                     total_reviews_loaded += 1
#                                     logging.info(f"Loaded {total_reviews_loaded}/{num_reviews} reviews")
                
#                                 except Exception as e:
#                                     logging.warning(f"Failed to extract a review: {e}")
#                                     continue
                
#                         # Scroll down
#                         driver.execute_script("arguments[0].scrollBy(0, arguments[0].scrollHeight);", scrollable_div)
#                         time.sleep(6)  # Wait for new reviews to load
                
#                         # Break if no new reviews are loaded
#                         if total_reviews_loaded == len(extracted_reviews) == num_reviews:
#                             logging.info(f"All {num_reviews} reviews have been loaded.")
#                             print("All reviews have been loaded.")
#                             break
                
#                 except Exception as e:
#                     logging.error(f"Failed during scrolling: {e}")
#                     driver.save_screenshot("scrolling_error.png")


#             except Exception as e:
#                 logging.warning(f"Failed to click on reviews or extract reviews: {e}")
#                 driver.save_screenshot(f"review_error_{idx}.png")
#                 continue

#         # Save data to CSV files
#         current_directory = os.getcwd()

#         # Save location data
#         location_file = os.path.join(current_directory, f"{business_name}_{city_name}_locations.csv")
#         pd.DataFrame(location_data).to_csv(location_file, index=False)
#         logging.info(f"Location details saved to {location_file}")

#         # Save review data
#         review_file = os.path.join(current_directory, f"{business_name}_{city_name}_reviews.csv")
#         pd.DataFrame(review_data).to_csv(review_file, index=False)
#         logging.info(f"Review details saved to {review_file}")

#     except Exception as e:
#         logging.error(f"An error occurred during scraping: {e}")
#         driver.save_screenshot("general_error.png")

#     finally:
#         logging.info("Closing WebDriver")
#         driver.quit()

In [4]:
# def scrape_google_maps_reviews(business_name, city_name, max_locations=5):
#     logging.info("Starting the scrape_google_maps_reviews function")
#     logging.info(f"Business name: {business_name}, City: {city_name}, Max locations: {max_locations}")

#     try:
#         options = webdriver.ChromeOptions()
#         options.add_argument("--disable-dev-shm-usage")
#         options.add_argument("--remote-debugging-port=9222")
#         driver = webdriver.Chrome(options=options)
#         driver.maximize_window()
#     except WebDriverException as e:
#         logging.error(f"Failed to initialize WebDriver: {e}")
#         return

#     try:
#         # Open Google Maps
#         logging.info("Opening Google Maps")
#         driver.get("https://www.google.com/maps")
#         wait = WebDriverWait(driver, 20)

#         # Search for the business
#         search_box = wait.until(EC.presence_of_element_located((By.ID, "searchboxinput")))
#         search_box.send_keys(f"{business_name} {city_name}")
#         search_box.send_keys(Keys.RETURN)
#         time.sleep(10)

#         # Locate business listings
#         listings = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.Nv2PK')))
#         if not listings:
#             logging.warning("No business listings found!")
#             return

#         business_links = []
#         for i, listing in enumerate(listings[:max_locations]):
#             link = listing.find_element(By.CSS_SELECTOR, 'a').get_attribute('href')
#             if "place" in link:
#                 business_links.append(link)

#         location_data = []
#         review_data = []

#         for idx, business_link in enumerate(business_links):
#             driver.get(business_link)
#             time.sleep(5)

#             # Capture location details
#             name = driver.find_element(By.CSS_SELECTOR, 'h1').text
#             address = driver.find_element(By.CSS_SELECTOR, '[data-item-id="address"]').text
#             overall_rating = driver.find_element(By.CSS_SELECTOR, 'div.F7nice span[aria-hidden="true"]').text
#             num_reviews_element = driver.find_element(By.CSS_SELECTOR, 'span[aria-label$="reviews"]')
#             num_reviews = int(''.join(filter(str.isdigit, num_reviews_element.get_attribute('aria-label'))))

#             location_id = f"LOC_{idx + 1}"
#             location_data.append({
#                 "Location ID": location_id,
#                 "Name": name,
#                 "Address": address,
#                 "Overall Rating": overall_rating,
#                 "Number of Reviews": num_reviews
#             })

#             # Extract reviews
#             reviews_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[@role='tab' and contains(@aria-label, 'Reviews')]")))
#             reviews_button.click()
#             time.sleep(10)

#             scrollable_div = driver.find_element(By.CSS_SELECTOR, 'div.m6QErb.DxyBCb.kA9KIf.dS8AEf')
#             prev_scroll_height = driver.execute_script("return arguments[0].scrollHeight;", scrollable_div)
#             extracted_reviews = set()

#             while len(extracted_reviews) < num_reviews:
#                 reviews = driver.find_elements(By.CSS_SELECTOR, '.jftiEf')
#                 for review in reviews:
#                     try:
#                         reviewer_name = review.find_element(By.CSS_SELECTOR, '.d4r55').text
#                         review_text = review.find_element(By.CSS_SELECTOR, '.MyEned').text if review.find_elements(By.CSS_SELECTOR, '.MyEned') else "No text"
#                         rating = review.find_element(By.CSS_SELECTOR, '.kvMYJc').get_attribute('aria-label')
#                         review_id = (reviewer_name, rating)  # Unique identifier

#                         if review_id not in extracted_reviews:
#                             review_data.append({
#                                 "Location ID": location_id,
#                                 "Reviewer Name": reviewer_name,
#                                 "Review": review_text,
#                                 "Rating": rating
#                             })
#                             extracted_reviews.add(review_id)

#                     except Exception as e:
#                         logging.warning(f"Failed to extract a review: {e}")

#                 # Scroll to load more
#                 driver.execute_script("arguments[0].scrollBy(0, arguments[0].scrollHeight);", scrollable_div)
#                 time.sleep(10)

#                 new_scroll_height = driver.execute_script("return arguments[0].scrollHeight;", scrollable_div)
#                 if new_scroll_height == prev_scroll_height:
#                     logging.info("No more reviews to load.")
#                     break
#                 prev_scroll_height = new_scroll_height

#             logging.info(f"Extracted {len(extracted_reviews)}/{num_reviews} reviews.")

#         # Save data
#         pd.DataFrame(location_data).to_csv(f"{business_name}_{city_name}_locations.csv", index=False)
#         pd.DataFrame(review_data).to_csv(f"{business_name}_{city_name}_reviews.csv", index=False)

#     except Exception as e:
#         logging.error(f"Error during scraping: {e}")
#     finally:
#         driver.quit()

In [5]:
# def get_locations_and_overall_ratings(business_name, city_name, max_locations=200):
#     logging.info("Getting locations and overall ratings")
#     try:
#         options = webdriver.ChromeOptions()
#         options.add_argument("--disable-dev-shm-usage")
#         driver = webdriver.Chrome(options=options)
#         driver.maximize_window()
#     except WebDriverException as e:
#         logging.error(f"Failed to initialize WebDriver: {e}")
#         return [], []

#     try:
#         driver.get("https://www.google.com/maps")
#         wait = WebDriverWait(driver, 20)

#         # Search for the business
#         logging.info("Searching for the business...")
#         search_box = wait.until(EC.presence_of_element_located((By.ID, "searchboxinput")))
#         search_box.send_keys(f"{business_name} {city_name}")
#         search_box.send_keys(Keys.RETURN)
#         time.sleep(10)

#         # Locate and scroll through business listings
#         logging.info("Scrolling through business listings to load all...")
        
#         scrollable_div = driver.find_element(By.CSS_SELECTOR, 'div.m6QErb.DxyBCb.kA9KIf.dS8AEf')
#         listings = []
#         retries = 0

#         while len(listings) < max_locations and retries < 5:
#             listings = driver.find_elements(By.CSS_SELECTOR, '.Nv2PK')
#             driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", scrollable_div)
#             time.sleep(3)
#             new_listings = driver.find_elements(By.CSS_SELECTOR, '.Nv2PK')
#             if len(new_listings) == len(listings):  # No new listings loaded
#                 retries += 1
#                 logging.info(f"No new listings loaded after scroll. Retry {retries}")
#             else:
#                 retries = 0

#         if not listings:
#             logging.warning("No business listings found!")
#             return [], []

#         logging.info(f"Found {len(listings)} listings.")

#         # Extract data from listings
#         business_links = []
#         location_data = []

#         for i, listing in enumerate(listings[:max_locations]):
#             try:
#                 link = listing.find_element(By.CSS_SELECTOR, 'a').get_attribute('href')
#                 if "place" in link:
#                     business_links.append(link)

#                     name = listing.find_element(By.CSS_SELECTOR, '.qBF1Pd').text
#                     address = listing.find_element(By.CSS_SELECTOR, '.W4Efsd').text
#                     overall_rating = listing.find_element(By.CSS_SELECTOR, '.MW4etd').text

#                     location_data.append({
#                         "Location ID": f"LOC_{i + 1}",
#                         "Name": name,
#                         "Address": address,
#                         "Overall Rating": overall_rating,
#                         "Link": link
#                     })
#             except Exception as e:
#                 logging.warning(f"Failed to extract data from a listing: {e}")

#         pd.DataFrame(location_data).to_csv(f"data/{business_name}_{city_name}_locations.csv", index=False)
#         logging.info("Location data saved successfully.")
#         return business_links, location_data

#     except Exception as e:
#         logging.error(f"Error during scraping: {e}")
#         return [], []
#     finally:
#         driver.quit()

In [7]:
# def get_locations_and_overall_ratings(business_name, city_name, max_locations=200):
#     logging.info("Getting locations and overall ratings")

#     # Initialize WebDriver
#     try:
#         options = webdriver.ChromeOptions()
#         options.add_argument("--disable-dev-shm-usage")
#         driver = webdriver.Chrome(options=options)
#         driver.maximize_window()
#     except WebDriverException as e:
#         logging.error(f"Failed to initialize WebDriver: {e}")
#         return [], []

#     try:
#         # Open Google Maps
#         driver.get("https://www.google.com/maps")
#         wait = WebDriverWait(driver, 20)

#         # Search for the business
#         logging.info("Searching for the business...")
#         search_box = wait.until(EC.presence_of_element_located((By.ID, "searchboxinput")))
#         search_box.send_keys(f"{business_name} {city_name}")
#         search_box.send_keys(Keys.RETURN)
#         time.sleep(10)

#         # Locate and scroll through business listings
#         logging.info("Scrolling through business listings to load all...")
#         scrollable_div = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div.m6QErb.DxyBCb.kA9KIf.dS8AEf')))
#         listings = []
#         retries = 0
#         last_scroll_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_div)

#         while len(listings) < max_locations and retries < 10:
#             # Scroll down in the listings container
#             driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", scrollable_div)
#             time.sleep(3)  # Short wait to allow loading of new listings

#             # Recalculate the listings
#             new_listings = driver.find_elements(By.CSS_SELECTOR, '.Nv2PK')
#             logging.info(f"Loaded {len(new_listings)} listings...")

#             # Check if new listings have been loaded
#             if len(new_listings) > len(listings):
#                 listings = new_listings
#                 retries = 0  # Reset retries since new data loaded
#             else:
#                 retries += 1
#                 logging.info(f"No new listings loaded after scroll. Retry: {retries}")

#             # Update scroll height
#             new_scroll_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_div)
#             if new_scroll_height == last_scroll_height:
#                 retries += 1
#                 logging.info("No change in scroll height. Retrying...")
#             else:
#                 last_scroll_height = new_scroll_height

#         if len(listings) == 0:
#             logging.warning("No business listings found!")
#             return [], []
#         else:
#             logging.info(f"Found {len(listings)} listings.")

#         # Extract data from listings
#         business_links = []
#         location_data = []

#         for i, listing in enumerate(listings[:max_locations]):
#             try:
#                 link = listing.find_element(By.CSS_SELECTOR, 'a').get_attribute('href')
#                 if "place" in link:
#                     business_links.append(link)

#                     # Extract location details
#                     name = listing.find_element(By.CSS_SELECTOR, '.qBF1Pd').text
#                     address = listing.find_element(By.CSS_SELECTOR, '.W4Efsd').text
#                     overall_rating = listing.find_element(By.CSS_SELECTOR, '.MW4etd').text

#                     location_data.append({
#                         "Location ID": f"LOC_{i + 1}",
#                         "Name": name,
#                         "Address": address,
#                         "Overall Rating": overall_rating,
#                         "Link": link
#                     })
#             except Exception as e:
#                 logging.warning(f"Failed to extract data from a listing: {e}")

#         # Save location data to CSV
#         pd.DataFrame(location_data).to_csv(f"data/{business_name}_{city_name}_locations.csv", index=False)
#         logging.info("Location data saved successfully.")
#         return business_links, location_data

#     except Exception as e:
#         logging.error(f"Error during scraping: {e}")
#         return [], []
#     finally:
#         driver.quit()

In [8]:
# get_locations_and_overall_ratings("Naturals", "Chennai", max_locations=100)

In [15]:
def get_overall_rating_and_review_count(business_name, address):
    logging.info(f"Scraping overall rating and review count for: {business_name}, located at: {address}")

    # Initialize WebDriver
    try:
        options = webdriver.ChromeOptions()
        options.add_argument("--disable-dev-shm-usage")
        options.add_argument("--remote-debugging-port=9222")
        driver = webdriver.Chrome(options=options)
        driver.maximize_window()
    except Exception as e:
        logging.error(f"Failed to initialize WebDriver: {e}")
        return {}

    try:
        # Open Google Maps
        driver.get("https://www.google.com/maps")
        wait = WebDriverWait(driver, 20)

        # Search for the business by name and address
        search_box = wait.until(EC.presence_of_element_located((By.ID, "searchboxinput")))
        search_box.clear()  # Clear the input box to avoid residual text
        search_box.send_keys(f"{business_name} {address}")
        search_box.send_keys(Keys.RETURN)
        time.sleep(5)

        # # Wait for the search results to load
        # try:
        #     wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.Nv2PK')))
        # except Exception as e:
        #     logging.error(f"Search results did not load: {e}")
        #     return {}

        # # Click on the first matching business
        # try:
        #     first_result = driver.find_element(By.CSS_SELECTOR, '.Nv2PK')
        #     first_result.click()
        # except Exception as e:
        #     logging.error(f"Failed to click on the first search result: {e}")
        #     return {}

        # time.sleep(10)  # Allow the business page to load

        # Extract overall rating and total number of reviews
        try:
            overall_rating = driver.find_element(By.CSS_SELECTOR, 'div.F7nice span[aria-hidden="true"]').text
            num_reviews_element = driver.find_element(By.CSS_SELECTOR, 'span[aria-label$="reviews"]')
            num_reviews = int(''.join(filter(str.isdigit, num_reviews_element.get_attribute('aria-label'))))
            logging.info(f"Overall Rating: {overall_rating}, Total Reviews: {num_reviews}")
        except Exception as e:
            logging.warning(f"Failed to extract overall rating or review count: {e}")
            overall_rating = "N/A"
            num_reviews = 0

        # Return the data
        result = {
            "Business Name": business_name,
            "Address": address,
            "Overall Rating": overall_rating,
            "Total Reviews": num_reviews,
        }
        logging.info(f"Data extracted: {result}")
        return result

    except Exception as e:
        logging.error(f"Error during scraping: {e}")
        return {}

    finally:
        driver.quit()

In [16]:
get_overall_rating_and_review_count("Naturals", "No F37A, 1st Floor, Spencer Plaza, Phase 2, Anna Salai, Triplicane, Chennai - 600002")

2024-12-13 17:39:17,968 - INFO - Scraping overall rating and review count for: Naturals, located at: No F37A, 1st Floor, Spencer Plaza, Phase 2, Anna Salai, Triplicane, Chennai - 600002
2024-12-13 17:39:29,261 - INFO - Overall Rating: 4.3, Total Reviews: 179
2024-12-13 17:39:29,262 - INFO - Data extracted: {'Business Name': 'Naturals', 'Address': 'No F37A, 1st Floor, Spencer Plaza, Phase 2, Anna Salai, Triplicane, Chennai - 600002', 'Overall Rating': '4.3', 'Total Reviews': 179}


{'Business Name': 'Naturals',
 'Address': 'No F37A, 1st Floor, Spencer Plaza, Phase 2, Anna Salai, Triplicane, Chennai - 600002',
 'Overall Rating': '4.3',
 'Total Reviews': 179}

In [23]:
def scrape_google_maps_reviews(business_name, city_name, max_locations: int):
    logging.info("Starting the scrape_google_maps_reviews function")
    logging.info(f"Business name: {business_name}, City: {city_name}, Max locations: {max_locations}")

    try:
        options = webdriver.ChromeOptions()
        options.add_argument("--disable-dev-shm-usage")
        options.add_argument("--remote-debugging-port=9222")
        driver = webdriver.Chrome(options=options)
        driver.maximize_window()
    except WebDriverException as e:
        logging.error(f"Failed to initialize WebDriver: {e}")
        return

    try:
        # Open Google Maps
        logging.info("Opening Google Maps")
        driver.get("https://www.google.com/maps")
        wait = WebDriverWait(driver, 20)

        # Search for the business
        search_box = wait.until(EC.presence_of_element_located((By.ID, "searchboxinput")))
        search_box.send_keys(f"{business_name} {city_name}")
        search_box.send_keys(Keys.RETURN)
        time.sleep(10)

        # Locate business listings
        listings = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.Nv2PK')))
        if not listings:
            logging.warning("No business listings found!")
            return

        business_links = []
        for i, listing in enumerate(listings[:max_locations]):
            link = listing.find_element(By.CSS_SELECTOR, 'a').get_attribute('href')
            if "place" in link:
                business_links.append(link)

        location_data = []
        review_data = []

        total_reviews_to_extract = 0

        
        for idx, business_link in enumerate(business_links):
            driver.get(business_link)
            time.sleep(5)

            # Capture location details
            name = driver.find_element(By.CSS_SELECTOR, 'h1').text
            address = driver.find_element(By.CSS_SELECTOR, '[data-item-id="address"]').text
            overall_rating = driver.find_element(By.CSS_SELECTOR, 'div.F7nice span[aria-hidden="true"]').text
            num_reviews_element = driver.find_element(By.CSS_SELECTOR, 'span[aria-label$="reviews"]')
            num_reviews = int(''.join(filter(str.isdigit, num_reviews_element.get_attribute('aria-label'))))

            location_id = f"LOC_{idx + 1}"
            location_data.append({
                "Location ID": location_id,
                "Name": name,
                "Address": address,
                "Overall Rating": overall_rating,
                "Number of Reviews": num_reviews
            })

            total_reviews_to_extract += num_reviews
            
            # Extract reviews
            reviews_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[@role='tab' and contains(@aria-label, 'Reviews')]")))
            reviews_button.click()
            time.sleep(5)

            scrollable_div = driver.find_element(By.CSS_SELECTOR, 'div.m6QErb.DxyBCb.kA9KIf.dS8AEf')
            extracted_reviews = {}
            retries = 0

            while len(extracted_reviews) < total_reviews_to_extract and retries < 10:
                reviews = driver.find_elements(By.CSS_SELECTOR, '.jftiEf')
                for review in reviews:
                    try:
                        reviewer_name = review.find_element(By.CSS_SELECTOR, '.d4r55').text
                        review_text = review.find_element(By.CSS_SELECTOR, '.MyEned').text if review.find_elements(By.CSS_SELECTOR, '.MyEned') else "No text"
                        rating = review.find_element(By.CSS_SELECTOR, '.kvMYJc').get_attribute('aria-label')

                        review_id = (reviewer_name, review_text)  # Unique identifier
                        if review_id not in extracted_reviews:
                            review_data.append({
                                "Location ID": location_id,
                                "Reviewer Name": reviewer_name,
                                "Review": review_text,
                                "Rating": rating
                            })
                            extracted_reviews[review_id] = True

                    except Exception as e:
                        logging.warning(f"Failed to extract a review: {e}")

                # Scroll down
                prev_height = driver.execute_script("return arguments[0].scrollHeight;", scrollable_div)
                driver.execute_script("arguments[0].scrollBy(0, arguments[0].scrollHeight);", scrollable_div)
                time.sleep(10)

                new_height = driver.execute_script("return arguments[0].scrollHeight;", scrollable_div)
                if new_height == prev_height:
                    retries += 1
                    logging.info(f"No new reviews loaded after scroll. Retry: {retries}")
                else:
                    retries = 0

            logging.info(f"Extracted {len(extracted_reviews)}/{total_reviews_to_extract} reviews.")

        # Save data
        
        pd.DataFrame(review_data).to_csv(f"data/{business_name}_{city_name}_reviews.csv", index=False)

    except Exception as e:
        logging.error(f"Error during scraping: {e}")
    finally:
        driver.quit()

In [24]:
get_locations_and_overall_ratings("Naturals", "Chennai", max_locations=100)

2024-12-13 17:11:41,962 - INFO - Getting locations and overall ratings
2024-12-13 17:11:48,509 - INFO - Searching for the business...
2024-12-13 17:11:58,643 - INFO - Scrolling through business listings to load all...
2024-12-13 17:12:08,709 - INFO - No new listings loaded after scroll. Retry 1
2024-12-13 17:12:13,733 - INFO - No new listings loaded after scroll. Retry 2
2024-12-13 17:12:18,756 - INFO - No new listings loaded after scroll. Retry 3
2024-12-13 17:12:23,781 - INFO - No new listings loaded after scroll. Retry 4
2024-12-13 17:12:28,804 - INFO - No new listings loaded after scroll. Retry 5
2024-12-13 17:12:28,806 - INFO - Found 9 listings.
2024-12-13 17:12:29,026 - INFO - Location data saved successfully.


(['https://www.google.com/maps/place/Naturals+Salon+Kundrathur/data=!4m7!3m6!1s0x3a528b744e0422e5:0xdfb6446b59f95f50!8m2!3d12.9995163!4d80.0972509!16s%2Fg%2F11shjjfg3v!19sChIJ5SIETnSLUjoRUF_5WWtEtt8?authuser=0&hl=en&rclk=1',
  'https://www.google.com/maps/place/Naturals+Salon/data=!4m7!3m6!1s0x3a525fccd1b0b8e1:0x39b8de9f4a0dd819!8m2!3d12.97281!4d80.138461!16s%2Fg%2F1hd_g7073!19sChIJ4biw0cxfUjoRGdgNSp_euDk?authuser=0&hl=en&rclk=1',
  'https://www.google.com/maps/place/Naturals+Signature+Salon/data=!4m7!3m6!1s0x3a525f51eeae09e9:0x5e3ab5480e607b8!8m2!3d12.9534393!4d80.1381887!16s%2Fg%2F11y400j3_k!19sChIJ6Qmu7lFfUjoRuAfmgFSr4wU?authuser=0&hl=en&rclk=1',
  'https://www.google.com/maps/place/Naturals+Signature+Salon/data=!4m7!3m6!1s0x3a525faac0d12d5d:0xe459872b62e82c7a!8m2!3d12.9423475!4d80.1418197!16s%2Fg%2F1ptz2p7t_!19sChIJXS3RwKpfUjoReizoYiuHWeQ?authuser=0&hl=en&rclk=1',
  'https://www.google.com/maps/place/Naturals+Salon/data=!4m7!3m6!1s0x3a525e061f40a0ef:0x828dbb6be8a0f865!8m2!3d12.9767